In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering, KMeans
from sklearn.neighbors import NearestNeighbors
import os.path as osp
import torch
import pickle

/home/yangliu/miniconda3/envs/k2/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/home/yangliu/miniconda3/envs/k2/lib/python3.9/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#depre

In [9]:
a2v_path = osp.expanduser('~/data/bert-vqa/a2v_clip_top3k.pt')
embedded: np.ndarray = torch.load(a2v_path).numpy()
shift = 2
embedded = embedded[shift: shift+300]
lbl2idx_path = osp.expanduser('~/data/vqav2/lbl2idx_separate_top3k.pkl')
with open(lbl2idx_path, "rb") as f:
    lbl2idx, idx2lbl = pickle.load(f)
# embedded = embedded[2:]
print(embedded.shape, embedded)

(300, 1024) [[ 0.11322021  0.44458008 -0.0604248  ... -0.19433594  0.10357666
   0.06130981]
 [ 0.06982422  0.4333496  -0.13623047 ... -0.23803711  0.06518555
   0.08843994]
 [ 0.3803711   0.14538574 -0.18713379 ...  0.04244995 -0.4338379
   0.4350586 ]
 ...
 [-0.12646484 -0.2208252  -0.14453125 ... -0.04483032  0.07000732
   0.18774414]
 [ 0.28198242  0.75097656 -0.12005615 ... -0.18151855 -0.03155518
   0.34521484]
 [ 0.18225098 -0.31933594 -0.07391357 ...  0.08581543 -0.24890137
   0.24804688]]


In [11]:
kmeans = KMeans(n_clusters=40)
kmeans.fit(embedded)

/home/yangliu/miniconda3/envs/k2/lib/python3.9/site-packages/sklearn/cluster/k_means_.py:443: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  centers, labels, n_iter = k_means_elkan(X, checked_sample_weight,
/home/yangliu/miniconda3/envs/k2/lib/python3.9/site-packages/sklearn/cluster/k_means_.py:443: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to s

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=40, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=None, tol=0.0001, verbose=0)

In [12]:
kmeans.cluster_centers_.shape

(40, 1024)

In [13]:
nn = NearestNeighbors(n_neighbors=1)
nn.fit(embedded)

NearestNeighbors(algorithm='auto', leaf_size=30, metric='minkowski',
                 metric_params=None, n_jobs=None, n_neighbors=1, p=2,
                 radius=1.0)

In [14]:
centroids = nn.kneighbors(kmeans.cluster_centers_, 2)[1][:,0]
centriods = centroids.sort()
print(centroids)

[  1  10  25  41  44  50  55  59  61  65  85  95  97 104 106 109 112 115
 120 130 132 146 150 153 158 160 165 171 174 211 219 247 248 261 262 268
 272 277 288 292]


In [15]:
centroids_lbl = [idx2lbl[int(centroid) + shift] for centroid in centroids]
print(centroids_lbl)

['1', 'brown', 'kitchen', 'kite', 'giraffe', 'wii', 'purple', 'table', 'food', 'banana', 'female', 'flowers', 'bird', 'bed', 'plane', 'car', 'cloudy', 'up', 'donut', 'horses', 'bike', 'not sure', 'knife', 'snowboard', 'couch', 'ocean', 'afternoon', 'tennis racket', 'toilet', 'field', 'mirror', 'white and black', 'carrot', 'luggage', 'oranges', 'noon', 'on', 'fire hydrant', 'striped', 'child']
